In [1]:
#settings 
# from settings.settings import data_settings

#imports
import pickle
import pandas as pd 
import numpy as np 
from datetime import datetime
import matplotlib as mpl 
import matplotlib.pyplot as plt

pd.options.display.max_columns = 500
pd.options.display.max_seq_items = 500
pd.options.display.max_rows = 500

In [2]:
# Loading data and making it ready
csv_path = "D:/dev/workspace/vscode/python/bos/data/csv"
tp_to_sl = 1
slatr = 5
tp_to_sl_s = [1,1.25,1.5,1.75,2,4,6]
date_time_format = "%Y-%m-%d %H:%M:%S"
name = "SP500m_H1_202009231400_202207082200_slatr_05"

df = pd.read_csv(f"{csv_path}/with_class/{name}.csv")
df.shape

(10134, 130)

In [3]:

train_starting_date = datetime(2018,2,1)
test_starting_date = datetime(2022,1,1)

df["DateTime"] = pd.to_datetime(
    df.pop("DateTime"), format=date_time_format)
df = df.loc[df.DateTime > train_starting_date ].reset_index(drop=True)

non_columns = ["CUMULATIVE_RETURN","DAILY_LOG_RETURN","KST","KST_SIG","NEGATIVE_VOLUME_INDEX","PSAR_DOWN","PSAR_DOWN_INDICATOR"
               ,"PSAR_UP","PSAR_UP_INDICATOR","VOLUME_PRICE_TREND","ROC","DAILY_RETURN","ULCER_INDEX"]
for column in non_columns:
    df.pop(column)
    
df = df.dropna().reset_index(drop=True)

df.shape

(9943, 117)

In [4]:

def print_nans(df):
    
    for column in df.columns:
        for j in range(df.shape[0]):
            # if(pd.isna(df[column].iloc[j])):
            if(df[column].iloc[j]==np.inf):
                print( column , ":",j)

# print_nans(df)

In [5]:
date_time = df.pop("DateTime")

buy = df[f"buy_{tp_to_sl}"].copy()
df["Buy"] = buy
sell = df[f"sell_{tp_to_sl}"].copy()
df["Sell"] = sell

for i in tp_to_sl_s:
    df.pop(f"buy_{i}")
    df.pop(f"sell_{i}")
    
df.pop(df.windowsizes.name)
df.pop(df.do_nothing.name)

do_nothings = []
for i in range(len(df)):
    if(buy.iloc[i] == 0 and sell.iloc[i] == 0):
        do_nothings.append(1)
    else:
        do_nothings.append(0)

df["do_nothing"] = do_nothings

relative_columns =[
        "BOLLINGER_HBAND",
        "BOLLINGER_LBAND",
        "BOLLINGER_MAVG",
        "DONCHIAN_CHANNEL_HBAND",
        "DONCHIAN_CHANNEL_LBAND",
        "DONCHIAN_CHANNEL_MBAND",
        "EMA_INDICATOR_012",
        "EMA_INDICATOR_024",
        "EMA_INDICATOR_048",
        "EMA_INDICATOR_072",
        "EMA_INDICATOR_096",
        "EMA_INDICATOR_120",
        "EMA_INDICATOR_144",
        "EMA_INDICATOR_168",
        "EMA_INDICATOR_192",
        "SMA_INDICATOR_012",
        "SMA_INDICATOR_024",
        "SMA_INDICATOR_048",
        "SMA_INDICATOR_072",
        "SMA_INDICATOR_096",
        "SMA_INDICATOR_120",
        "SMA_INDICATOR_144",
        "SMA_INDICATOR_168",
        "SMA_INDICATOR_192",
        # "Open",
        # "High",
        # "Low",
        "ICHIMOKU_A",
        "ICHIMOKU_B",
        "ICHIMOKU_BASE_LINE",
        "ICHIMOKU_CONVERSION_LINE",
        "KELTNER_CHANNEL_HBAND",
        "KELTNER_CHANNEL_LBAND",
        "KELTNER_CHANNEL_MBAND",
        "NEGATIVE_VOLUME_INDEX",
        "VOLUME_WEIGHTED_AVERAGE_PRICE"
        "PSAR_UP"
        "PSAR_DOWN"
    ]

for column in relative_columns:
    if column in df.columns:
        df[f"{column}_C"] = (df[column] / df.Close)*100

# for column in df.columns:
#     if(column not in ["Close","Open","High","Low","Buy","Sell","do_nothing"]):
        
#         new_column = df[column].div(df.Close).mul(100)
#         new_column.name =f"{column}_C"
#         df = pd.concat([df,new_column],axis=1)
        
        

In [6]:
df.head()

,Open,High,Low,Close,TickVol,Spread,PPO,HIST,PPO_SIGNAL,RSI,STOCHRSI,STOCHRSI_D,STOCHRSI_K,TSI,BOLLINGER_HBAND,BOLLINGER_HBAND_INDICATOR,BOLLINGER_LBAND,BOLLINGER_LBAND_INDICATOR,BOLLINGER_MAVG,BOLLINGER_PBAND,BOLLINGER_WBAND,DPO,MACD,MACD_DIFF,MACD_SIGNAL,STC,TRIX,PVO,PVO_HIST,PVO_SIGNAL,CCI,STOCH,STOCH_SIGNAL,ULTIMATE_OSCILLATOR,WILLIAMS_R,AVERAGE_TRUE_RANGE,DONCHIAN_CHANNEL_HBAND,DONCHIAN_CHANNEL_LBAND,DONCHIAN_CHANNEL_MBAND,DONCHIAN_CHANNEL_PBAND,DONCHIAN_CHANNEL_WBAND,KELTNER_CHANNEL_HBAND,KELTNER_CHANNEL_HBAND_INDICATOR,KELTNER_CHANNEL_LBAND,KELTNER_CHANNEL_LBAND_INDICATOR,KELTNER_CHANNEL_MBAND,KELTNER_CHANNEL_PBAND,KELTNER_CHANNEL_WBAND,ADX,ADX_NEG,ADX_POS,VORTEX_INDICATOR_NEG,VORTEX_INDICATOR_POS,EASE_OF_MOVEMENT,SMA_EASE_OF_MOVEMENT,ACC_DIST_INDEX,CHAIKIN_MONEY_FLOW,MONEY_FLOW_INDEX,VOLUME_WEIGHTED_AVERAGE_PRICE,FORCE_INDEX,ON_BALANCE_VOLUME,AWESOME_OSCILLATOR,ICHIMOKU_A,ICHIMOKU_B,ICHIMOKU_BASE_LINE,ICHIMOKU_CONVERSION_LINE,MASS_INDEX,AROON_DOWN,AROON_INDICATOR,AROON_UP,EMA_INDICATOR_012,EMA_INDICATOR_024,EMA_INDICATOR_048,EMA_INDICATOR_072,EMA_INDICATOR_096,EMA_INDICATOR_120,EMA_INDICATOR_144,EMA_INDICATOR_168,EMA_INDICATOR_192,SMA_INDICATOR_012,SMA_INDICATOR_024,SMA_INDICATOR_048,SMA_INDICATOR_072,SMA_INDICATOR_096,SMA_INDICATOR_120,SMA_INDICATOR_144,SMA_INDICATOR_168,SMA_INDICATOR_192,Day_sin,Day_cos,One_Year_sin,One_Year_cos,Two_Year_sin,Two_Year_cos,Three_Year_sin,Three_Year_cos,Four_Year_sin,Four_Year_cos,Five_Year_sin,Five_Year_cos,Buy,Sell,do_nothing,BOLLINGER_HBAND_C,BOLLINGER_LBAND_C,BOLLINGER_MAVG_C,DONCHIAN_CHANNEL_HBAND_C,DONCHIAN_CHANNEL_LBAND_C,DONCHIAN_CHANNEL_MBAND_C,EMA_INDICATOR_012_C,EMA_INDICATOR_024_C,EMA_INDICATOR_048_C,EMA_INDICATOR_072_C,EMA_INDICATOR_096_C,EMA_INDICATOR_120_C,EMA_INDICATOR_144_C,EMA_INDICATOR_168_C,EMA_INDICATOR_192_C,SMA_INDICATOR_012_C,SMA_INDICATOR_024_C,SMA_INDICATOR_048_C,SMA_INDICATOR_072_C,SMA_INDICATOR_096_C,SMA_INDICATOR_120_C,SMA_INDICATOR_144_C,SMA_INDICATOR_168_C,SMA_INDICATOR_192_C,ICHIMOKU_A_C,ICHIMOKU_B_C,ICHIMOKU_BASE_LINE_C,ICHIMOKU_CONVERSION_LINE_C,KELTNER_CHANNEL_HBAND_C,KELTNER_CHANNEL_LBAND_C,KELTNER_CHANNEL_MBAND_C
0,3404.5,3407.6,3403.9,3404.4,450,6,0.312701,0.014088,0.298614,64.128182,0.673075,0.712251,0.710583,26.022180,3420.267810,0.0,3358.102190,0.0,3389.185,0.744749,1.834235,5.215,10.593327,0.492307,10.101020,99.999851,0.044682,-25.898466,-5.490163,-20.408303,79.050976,87.750000,87.780806,49.290090,-12.250000,9.287120,3409.3,3356.1,3397.20,0.797521,0.711411,3397.191667,1.0,3380.211667,0.0,3388.701667,1.424519,0.501077,19.211439,16.901606,23.152054,0.691429,1.288163,2.877778e+05,1.738947e+06,11903.651885,0.120729,77.869481,3394.785463,1928.688715,28367,24.988529,3389.950,3359.25,3382.7,3397.20,21.757074,24.0,48.0,72.0,3398.273577,3388.934768,3377.999963,3370.780969,3364.493758,3358.975567,3354.255566,3350.275305,3346.931713,3398.273577,3388.934768,3377.999963,3370.780969,3364.493758,3358.975567,3354.255566,3350.275305,3346.931713,-5.064574e-08,1.0,-0.570070,-0.821596,-0.954357,0.298667,0.948794,0.315897,-0.592171,0.805812,0.906669,-0.421843,0,1,0,100.466097,98.640060,99.553078,100.143931,98.581248,99.788509,99.820044,99.545728,99.224532,99.012483,98.827804,98.665714,98.527070,98.410155,98.311941,99.820044,99.545728,99.224532,99.012483,98.827804,98.665714,98.527070,98.410155,98.311941,99.575549,98.673775,99.362590,99.788509,99.788264,99.289498,99.538881
1,3404.4,3404.9,3400.6,3404.4,575,6,0.303854,0.004192,0.299662,64.128182,0.522174,0.697259,0.644632,26.388521,3420.341125,0.0,3362.308875,0.0,3391.325,0.725306,1.711197,4.675,10.297353,0.157067,10.140287,99.999925,0.045045,-27.368757,-5.568363,-21.800394,65.335392,85.240964,87.126530,52.919949,-14.759036,8.930897,3409.3,3360.8,3397.20,0.797521,0.711075,3398.641667,1.0,3382.781667,0.0,3390.711667,1.363073,0.467748,18.426344,18.959654,22.355831,0.783406,1.260262,-2.243478e+06,1.332573e+06,12344.930955,0.200752,71.099851,3398.267837,1653.161755,28942,24.153235,3392.200,3359.25,3382.7,3401.70,21.856070,

In [7]:
moving_averages = [        
    "EMA_INDICATOR_192" ,
    "SMA_INDICATOR_192" ,
    "EMA_INDICATOR_168"  ,
    "SMA_INDICATOR_168"  ,
    "EMA_INDICATOR_144"  ,
    "SMA_INDICATOR_144"  ,
    "EMA_INDICATOR_120"  ,
    "SMA_INDICATOR_120"  ,
    "EMA_INDICATOR_096"  ,
    "SMA_INDICATOR_096"  ,
    "EMA_INDICATOR_072"  ,
    "SMA_INDICATOR_072"  ,
    "EMA_INDICATOR_048"  ,
    "SMA_INDICATOR_048"  ,
    "EMA_INDICATOR_024"  ,
    "SMA_INDICATOR_024"  ,
    "EMA_INDICATOR_012"  ,
    "SMA_INDICATOR_012"  ,
]

for column in moving_averages[:]:
    moving_averages.remove(column)
    
    for i in moving_averages[:]:
        new_column = df[f"{column}"] / df[f"{i}"]
        new_column.name = f"{column}/{i}"
        df = pd.concat([df,new_column],axis=1)

# important_columns = [        
#     "CUMULATIVE_RETURN",
#     "NEGATIVE_VOLUME_INDEX_C",
#     "AVERAGE_TRUE_RANGE",
#     "SMA_INDICATOR_192_C",
#     "EMA_INDICATOR_192_C",
#     "SMA_INDICATOR_144_C",
#     "EMA_INDICATOR_144_C",
#     "SMA_INDICATOR_120_C",
#     "EMA_INDICATOR_120_C",
#     "SMA_INDICATOR_096_C",
#     "EMA_INDICATOR_096_C",
#     "MASS_INDEX",
#     "Day_sin",
#     "ADX",
#     "KELTNER_CHANNEL_WBAND",
#     "ULCER_INDEX",
#     "TickVol",
#     "DONCHIAN_CHANNEL_WBAND",
#     "PPO",
#     "BOLLINGER_WBAND",
#     "KST_SIG",
#     "CHAIKIN_MONEY_FLOW",
#     "HIST",
#     "PPO_SIGNAL",
#     "ICHIMOKU_B_C",
#     "PVO",
#     "TSI"
# ]

# for column in important_columns[:]:
#     important_columns.remove(column)
    
#     for i in important_columns[:]:
#         new_column = df[f"{column}"] / df[f"{i}"]
#         new_column.name = f"{column}/{i}"
#         df = pd.concat([df,new_column],axis=1)



In [8]:
split_year = 2022
split_month = 1
split_day = 1
date_time_format = "%Y-%m-%d %H:%M:%S"
df.fillna(0,inplace=True)


# df["DateTime"] = pd.to_datetime(
#     df.pop("DateTime"), format=date_time_format)
df["DateTime"] = pd.to_datetime(date_time, format=date_time_format)
df.replace(np.inf,1e10,inplace=True)

train_df = df.loc[df.DateTime < datetime(
    split_year, split_month, split_day)].reset_index(drop=True)
test_df = df.loc[df.DateTime >= datetime(
    split_year, split_month, split_day)].reset_index(drop=True)


In [9]:
train_close = train_df.pop("Close")
test_close = test_df.pop("Close")

train_open = train_df.pop("Open")         #.Open[:]    
test_open = test_df.pop("Open")           #.Open[:]
        
train_high = train_df.pop("High")         #.High[:]    
test_high = test_df.pop("High")           #.High[:]
        
train_low = train_df.pop("Low")           #.Low[:]     
test_low = test_df.pop("Low")             #.Low[:]         

train_df_rc = []
test_df_rc = []

relative_columns= [
        "BOLLINGER_HBAND",
        "BOLLINGER_LBAND",
        "BOLLINGER_MAVG",
        "DONCHIAN_CHANNEL_HBAND",
        "DONCHIAN_CHANNEL_LBAND",
        "DONCHIAN_CHANNEL_MBAND",
        "EMA_INDICATOR_012",
        "EMA_INDICATOR_024",
        "EMA_INDICATOR_048",
        "EMA_INDICATOR_072",
        "EMA_INDICATOR_096",
        "EMA_INDICATOR_120",
        "EMA_INDICATOR_144",
        "EMA_INDICATOR_168",
        "EMA_INDICATOR_192",
        "SMA_INDICATOR_012",
        "SMA_INDICATOR_024",
        "SMA_INDICATOR_048",
        "SMA_INDICATOR_072",
        "SMA_INDICATOR_096",
        "SMA_INDICATOR_120",
        "SMA_INDICATOR_144",
        "SMA_INDICATOR_168",
        "SMA_INDICATOR_192",
        # "Open",
        # "High",
        # "Low",
        "ICHIMOKU_A",
        "ICHIMOKU_B",
        "ICHIMOKU_BASE_LINE",
        "ICHIMOKU_CONVERSION_LINE",
        "KELTNER_CHANNEL_HBAND",
        "KELTNER_CHANNEL_LBAND",
        "KELTNER_CHANNEL_MBAND",
        "NEGATIVE_VOLUME_INDEX",
        "VOLUME_WEIGHTED_AVERAGE_PRICE"
        "PSAR_UP"
        "PSAR_DOWN"
    ]

for column in relative_columns:
    if column in train_df.columns:
        train_df_rc.append(train_df.pop(column))
        test_df_rc.append(test_df.pop(column))

In [10]:
test_buy = test_df.pop(test_df.Buy.name)
test_nothing= test_df.pop(test_df.do_nothing.name)
test_sell = test_df.pop(test_df.Sell.name)

train_buy = train_df.pop(train_df.Buy.name)
train_nothing = train_df.pop(train_df.do_nothing.name)
train_sell = train_df.pop(train_df.Sell.name)

train_date_time = train_df.pop(train_df.DateTime.name)
test_date_time = test_df.pop(test_df.DateTime.name)

# X_train = (train_df - train_df.mean())/train_df.std()
# X_test = (test_df - train_df.mean())/train_df.std()

X_train = train_df
X_test = test_df

# y_train = pd.concat([train_sell,(train_buy + train_sell).subtract(1).abs(),train_buy] , axis = 1).to_numpy()
# y_test = pd.concat([test_sell,(test_buy+test_sell).subtract(1).abs(),test_buy] , axis = 1).to_numpy()


# y_train = []
# for buy,nothing,sell in zip(train_buy,train_nothing,train_sell):
#     if(buy == 1):
#         y_train.append(0)
#     elif(nothing == 1):
#         y_train.append(1)
#     elif(sell == 1):
#         y_train.append(2)

# y_test = []
# for buy,nothing,sell in zip(test_buy,test_nothing,test_sell):
#     if(buy == 1):
#         y_test.append(0)
#     elif(nothing == 1):
#         y_test.append(1)
#     elif(sell == 1):
#         y_test.append(2)     
        
print(X_train.shape,X_test.shape)



(7029, 249) (2914, 249)


In [11]:
y_train = pd.concat([train_sell,train_sell.subtract(1).abs()] , axis = 1).to_numpy()
y_test = pd.concat([test_sell,test_sell.subtract(1).abs()] , axis = 1).to_numpy()

In [24]:
sins = ['Day_sin', 'Day_cos', 'One_Year_sin',
       'One_Year_cos', 'Two_Year_sin', 'Two_Year_cos', 'Three_Year_sin',
       'Three_Year_cos', 'Four_Year_sin', 'Four_Year_cos', 'Five_Year_sin',
       'Five_Year_cos']

for i in sins:
    if i in X_train.columns:
        X_train.pop(i)
        X_test.pop(i)
        
    
X_train.shape[1] == X_test.shape[1]

True

In [64]:
y_train

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]], dtype=int64)

In [ ]:

a = f"D:/dev/workspace/vscode/python/bos/model/models/XGBOOST/{name}_{tp_to_sl:02}_{10000}/std.pickle"
with open(a , "rb") as f:
    b = pickle.load(f)


In [12]:
#MODEL
from xgboost import XGBRFClassifier
from sklearn.metrics import log_loss , precision_score

size = 2000
max_depth=12
# learning_rate = 1
# max_bin=512

model = XGBRFClassifier(
                        n_estimators=size,
                        max_depth=max_depth,
                        max_leaves=0,
                        # max_bin=max_bin,
                        subsample=0.9,
                        colsample_bynode=0.2,
                        grow_policy='depthwise',
                        # learning_rate=learning_rate,
                        verbosity = 1,
                        booster='gbtree',
                        tree_method = "gpu_hist",
                        n_jobs=0,
                        gamma=0,
                        min_child_weight=1,
                        sampling_method='gradient_based',
                        colsample_bytree=1,
                        missing=np.nan,
                        colsample_bylevel=1,
                        reg_alpha=0,
                        base_score=0.5,
                        random_state=23,
                        num_parallel_tree=size*5,
                        gpu_id=0,
                        predictor='auto',
                        eval_metric = precision_score,
                        max_delta_step=0,
                        # num_class = 2,
                        # objective='multi:softprob',
                        )

In [13]:
model

XGBRFClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bynode=0.2, colsample_bytree=1,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=<function precision_score at 0x000001BE8EF94790>,
                gamma=0, gpu_id=0, grow_policy='depthwise',
                importance_type=None, interaction_constraints=None,
                max_bin=None, max_cat_to_onehot=None, max_delta_step=0,
                max_depth=12, max_leaves=0, min_child_weight=1, missing=nan,
                monotone_constraints=None, n_estimators=2000, n_jobs=0,
                num_parallel_tree=10000, objective='binary:logistic',
                predictor='auto', random_state=23, reg_alpha=0,
                sampling_method='gradient_based', ...)

In [14]:
model.fit(
            X_train,
            y_train,
            # eval_set = [(X_test , y_test)]
        )


XGBRFClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bynode=0.2, colsample_bytree=1,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=<function precision_score at 0x000001BE8EF94790>,
                gamma=0, gpu_id=0, grow_policy='depthwise',
                importance_type=None, interaction_constraints='', max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=12,
                max_leaves=0, min_child_weight=1, missing=nan,
                monotone_constraints='()', n_estimators=2000, n_jobs=0,
                num_parallel_tree=10000, objective='binary:logistic',
                predictor='auto', random_state=23, reg_alpha=0,
                sampling_method='gradient_based', ...)

In [ ]:
# from xgboost import XGBRFClassifier

# model_data_name = "EURUSD_H1_201801020000_202206210600_01_10000"
# path = f'D:/dev/workspace/vscode/python/bos/model/models/XGBOOST/{model_data_name}'
# model = XGBRFClassifier()
# model.load_model(f"{path}/model01.json")

In [ ]:
from xgboost import plot_importance
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams["figure.figsize"]=(20,40)

plot_importance(model,max_num_features= 100)

df[["ACC_DIST_INDEX","ON_BALANCE_VOLUME","NEGATIVE_VOLUME_INDEX_C","CUMULATIVE_RETURN","EMA_INDICATOR_192/EMA_INDICATOR_168",
    "AVERAGE_TRUE_RANGE","EMA_INDICATOR_168","PPO","MASS_INDEX","TickVol","KELTNER_CHANNEL_WBAND","HIST","ADX","PSAR_DOWN"]]

In [15]:
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

model.save_model("./tmp/model.json")
p_train = model.predict(X_train)
p_test = model.predict(X_test)

rep_train = classification_report(y_train,p_train)
rep_test = classification_report(y_test,p_test)

print(rep_train)
print(rep_test)


c:\Users\a_aba\anaconda3\envs\fpp\lib\site-packages\xgboost\sklearn.py:722: UserWarning: eval_metric is not saved in Scikit-Learn meta.
  warnings.warn(str(k) + ' is not saved in Scikit-Learn meta.', UserWarning)


              precision    recall  f1-score   support

           0       1.00      0.93      0.96       872
           1       0.99      1.00      1.00      6157

   micro avg       0.99      0.99      0.99      7029
   macro avg       0.99      0.97      0.98      7029
weighted avg       0.99      0.99      0.99      7029
 samples avg       0.99      0.99      0.99      7029

              precision    recall  f1-score   support

           0       0.60      0.07      0.13       495
           1       0.84      0.99      0.91      2419

   micro avg       0.83      0.84      0.83      2914
   macro avg       0.72      0.53      0.52      2914
weighted avg       0.80      0.84      0.78      2914
 samples avg       0.83      0.84      0.83      2914



c:\Users\a_aba\anaconda3\envs\fpp\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\a_aba\anaconda3\envs\fpp\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
for i in range(y_train.shape[0]):
    if y_train[i][1]== 1:
        print(i)

              precision    recall  f1-score   support

           0       0.94      0.35      0.51      5261
           1       0.68      0.98      0.80     13749
           2       0.81      0.41      0.55      5656

    accuracy                           0.71     24666
   macro avg       0.81      0.58      0.62     24666
weighted avg       0.76      0.71      0.68     24666

              precision    recall  f1-score   support

           0       0.31      0.15      0.21       557
           1       0.58      0.90      0.71      1614
           2       0.35      0.05      0.09       689

    accuracy                           0.55      2860
   macro avg       0.42      0.37      0.33      2860
weighted avg       0.48      0.55      0.46      2860

--------------------------------------------------------------
           0       0.75      0.82      0.78     40462
           1       0.86      0.53      0.66     16535
           2       0.75      0.80      0.77     42414

    accuracy                           0.76     99411
   macro avg       0.79      0.72      0.74     99411
weighted avg       0.77      0.76      0.76     99411

              precision    recall  f1-score   support

           0       0.43      0.70      0.54      5218
           1       0.13      0.06      0.08      1322
           2       0.52      0.32      0.39      6283

    accuracy                           0.45     12823
   macro avg       0.36      0.36      0.34     12823
weighted avg       0.45      0.45      0.42     1282


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

lgbm_model = LGBMClassifier(
    n_estimators = 500,
    objective = "multiclass",
    random_state = 23,
    max_depth=12,
    learning_rate = .01,
    num_leaves = 31,
)

lgbm_model.fit(
    X = X_train,
    y = y_train,
    eval_set=(X_test,y_test)
    
)

p_train_lgbm = lgbm_model.predict(X_train)
p_test_lgbm = lgbm_model.predict(X_test)

acc_train_lgbm = accuracy_score(p_train_lgbm,y_train)
acc_test_lgbm = accuracy_score(p_test_lgbm,y_test)

rep_train_lgbm = classification_report(y_train,p_train_lgbm)
rep_test_lgbm = classification_report(y_test,p_test_lgbm)

con_train_lgbm = confusion_matrix(y_train,p_train_lgbm)
con_test_lgbm = confusion_matrix(y_test,p_test_lgbm)

print(rep_train_lgbm)
print(rep_test_lgbm)


In [21]:
a = []
for i in range(p_test.shape[0]):
    a.append(p_test[i][0])
    

62.0

In [23]:
df = X_test.copy()
# df = df * train_df.std() + train_df.mean()

df["DateTime"]= pd.to_datetime(test_date_time)
df=df.set_index(pd.DatetimeIndex(df["DateTime"]))

df["Signal"] = a
df["Close"] = test_close.tolist() 
df["High"] = test_high.tolist() 
df["Open"] = test_open.tolist() 
df["Low"] = test_low.tolist() 

df["Signal"].astype(np.float32)
df["Signal"] = df["Signal"]
# df["Class"] = [i * 1e-4 for i in y_test]
df = df

In [ ]:
import ta
df["VWAP"]=ta.volume.volume_weighted_average_price(df.High, df.Low, df.Close, df.TickVol)
df["EMA"]=ta.trend.ema_indicator(df.Close, window=100)

emasignal = [0]*len(df)
backcandles = 6

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.EMA[i]:
            dnt=0
        if df.Low[i]<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        #print("!!!!! check trend loop !!!!")
        emasignal[row]=3
    elif upt==1:
        emasignal[row]=2
    elif dnt==1:
        emasignal[row]=1

df['EMASignal'] = emasignal

VWAPsignal = [0]*len(df)
backcandles = 3

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.VWAP[i]:
            dnt=0
        if df.Low[i]<=df.VWAP[i]:
            upt=0
    if upt==1 and dnt==1:
        #print("!!!!! check trend loop !!!!")
        VWAPsignal[row]=3
    elif upt==1:
        VWAPsignal[row]=2
    elif dnt==1:
        VWAPsignal[row]=1

df['VWAPSignal'] = VWAPsignal

def TotalSignal(l):
    myclosedistance = 100
    if (df.EMASignal[l]==2 and df.VWAPSignal[l]==2):#and df.EngulfingSignal[l]==2 
        # and min(abs(df.VWAP[l]-df.High[l]),abs(df.VWAP[l]-df.Low[l]))<=myclosedistance):
            return 2e-4
    if (df.EMASignal[l]==1 and df.VWAPSignal[l]==1):#and df.EngulfingSignal[l]==1 
        # and min(abs(df.VWAP[l]-df.High[l]),abs(df.VWAP[l]-df.Low[l]))<=myclosedistance):
            return 1e-4
    else:
        return 0
        
TotSignal = [0]*len(df)
for row in range(0, len(df)): #careful backcandles used previous cell
    TotSignal[row] = TotalSignal(row)
df['TotalSignal'] = TotSignal

import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-3
    elif x['TotalSignal']==2:
        return x['Low']-1e-3
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)



62.0

In [47]:
from backtesting import Strategy

def Signal():
    return df.Signal

def Class():
    return df.Class

def vwap_signal():
    return df.TotalSignal


class MyStrategy(Strategy):


    def init(self):
        super().init()
        self.c = 0
        self.initsize = .02
        self.mysize = self.initsize
        self.signal1 = self.I(Signal)
        # self.class1 = self.I(Class)
        # self.vwap_signal = self.I(vwap_signal)
        

    def next(self):
        super().next()
        
        sl_to_atr = 5 * 1
        tp_to_atr = 1 * sl_to_atr
        
        # if(self.signal1 != 1 and len(self.trades) == 0 and len(self.closed_trades) > 0 and self.closed_trades[-1].pl < 0):
        #     if self.mysize < 128_000:
        #         self.mysize = int(self.mysize * 1.1)
        #         if self.mysize > 128_000:
        #             self.mysize = 128_000
        #     else:
        #         self.c += 1

        # elif len(self.closed_trades) > 0 and self.closed_trades[-1].pl > 0:
        #     if self.c > 0:
        #         self.c -= 1
        #     else:
        #         self.mysize = self.initsize
        
        if (
            # self.vwap_signal == 2e-4  or 
            (self.signal1 == 0 * 1e-4 and
            # self.data.EMA_INDICATOR_48_C <= 1   and
            # self.data.EMA_INDICATOR_096_C <= 1   and
            # self.data.EMA_INDICATOR_012 <= 1    and
            0)):
            
            sl1 = self.data.Close[-1] - (sl_to_atr * min(self.data.AVERAGE_TRUE_RANGE[-1],df.AVERAGE_TRUE_RANGE.mean())) #- self.data.Spread[-1]#*1e-4
            tp1 = self.data.Close[-1] + (tp_to_atr * min(self.data.AVERAGE_TRUE_RANGE[-1],df.AVERAGE_TRUE_RANGE.mean())) #+ self.data.Spread[-1]#*1e-4
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        if (
            # self.vwap_signal == 1e-4 or 
            (self.signal1 == 1 and
            # self.data.EMA_INDICATOR_48_C >= 100   and
            # self.data.EMA_INDICATOR_96_C >= 1   and
            # self.data.EMA_INDICATOR_8_C >= 1    and
            1)
                ):
            
            sl1=self.data.Close[-1] + (sl_to_atr * min(self.data.AVERAGE_TRUE_RANGE[-1],df.AVERAGE_TRUE_RANGE.mean())) #+ self.data.Spread[-1]#*1e-4
            tp1=self.data.Close[-1] - (tp_to_atr * min(self.data.AVERAGE_TRUE_RANGE[-1],df.AVERAGE_TRUE_RANGE.mean())) #- self.data.Spread[-1]#*1e-4
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
            
            


from backtesting import Backtest

bt=Backtest(df, MyStrategy, cash=10_000, margin=1/100, commission=.002,hedging=True)
stat=bt.run()
# stat.to_csv(f"{path}/stat.csv")
stat

Start                     2022-01-03 01:00:00
End                       2022-07-07 18:00:00
Duration                    185 days 17:00:00
Exposure Time [%]                   10.535347
Equity Final [$]                 26914.199034
Equity Peak [$]                   30601.67252
Return [%]                          169.14199
Buy & Hold Return [%]              -18.653629
Return (Ann.) [%]                  543.608616
Volatility (Ann.) [%]              664.857625
Sharpe Ratio                         0.817632
Sortino Ratio                       43.507931
Calmar Ratio                        45.042245
Max. Drawdown [%]                  -12.068862
Avg. Drawdown [%]                    -1.91044
Max. Drawdown Duration        9 days 18:00:00
Avg. Drawdown Duration        0 days 16:00:00
# Trades                                   62
Win Rate [%]                        93.548387
Best Trade [%]                       2.241917
Worst Trade [%]                     -1.938616
Avg. Trade [%]                    

In [38]:
bt.plot(results=None,
        filename=None,
        plot_width=None,
        plot_equity=True,
        plot_return=False,
        plot_pl=True,
        plot_volume=False,
        plot_drawdown=False,
        smooth_equity=False,
        relative_equity=True,
        superimpose=True,
        resample=True,
        reverse_indicators=False,
        show_legend=True,
        open_browser=True)

Row(id='1418', ...)

In [41]:
from xgboost import plot_importance ,plot_tree, to_graphviz , Booster
import pathlib

# check if path is available
path = f'D:/dev/workspace/vscode/python/bos/model/models/XGBOOST/{name}_{tp_to_sl:02}'
pathlib.Path(path).mkdir(parents=True,exist_ok=True)

#SAVE

with open(f"{path}/train{tp_to_sl:02}_report.txt","w") as f:
    f.write(rep_train)

with open(f"{path}/test{tp_to_sl:02}_report.txt","w") as f:
    f.write(rep_test) 
    
model.save_model(f"{path}/model{tp_to_sl:02}.json")

train_df.mean().to_pickle(f"{path}/mean.pickle")
train_df.std().to_pickle(f"{path}/std.pickle")

train_df.to_pickle(f"{path}/train_df.pickle")
test_df.to_pickle(f"{path}/test_df.pickle")
    
#LOAD    

# model = XGBRFClassifier()
# model.load_model(f"{path}/model.json")


#Drow model
graph = to_graphviz(model)
graph.render(filename=f"{path}/model")

c:\Users\a_aba\anaconda3\envs\fpp\lib\site-packages\xgboost\sklearn.py:722: UserWarning: eval_metric is not saved in Scikit-Learn meta.
  warnings.warn(str(k) + ' is not saved in Scikit-Learn meta.', UserWarning)


'D:\\dev\\workspace\\vscode\\python\\bos\\model\\models\\XGBOOST\\SP500m_H1_202009231400_202207082200_slatr_05_01\\model.pdf'

In [44]:
1/(1+np.exp(-1*0.000999998767))

0.5002499996709168

In [ ]:
# Creating generators

# window_size = data_settings.window_size
# batch_size : int = 256
# num_features : int = 97
# num_classes : int = 3

# def x_train_generator():
#     for i in range(window_size,len(train_df)+1):
#         window=train_df[i-window_size:i]
#         yield window.to_numpy()

# def x_test_generator():
#     for i in range(window_size,len(test_df)+1):
#         window=test_df[i-window_size:i]
#         yield window.to_numpy()    
        
# def y_train_generator():
#     for i in range(window_size,train_labels.shape[0]+1):
#         yield train_labels[i-1]

# def y_test_generator():
#     for i in range(window_size,test_labels.shape[0]+1):
#         yield test_labels[i-1]
        
# x_train_ds = tf.data.Dataset.from_generator(x_train_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(window_size,num_features), dtype=tf.float64)
#                                             )
#                                         ).batch(batch_size)

# x_test_ds = tf.data.Dataset.from_generator(x_test_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(window_size,num_features), dtype=tf.float64)
#                                             )
#                                         ).batch(batch_size)
# y_train_ds = tf.data.Dataset.from_generator(y_train_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(num_classes), dtype=tf.float32)
#                                             )
#                                         ).batch(batch_size)
# y_test_ds = tf.data.Dataset.from_generator(y_test_generator,
#                                         output_signature=(
#                                             tf.TensorSpec(shape=(num_classes), dtype=tf.float32)
#                                             )
#                                         ).batch(batch_size)

# train_ds = tf.data.Dataset.zip((x_train_ds,y_train_ds))
# test_ds = tf.data.Dataset.zip((x_test_ds,y_test_ds))

# def compile_and_fit(model:keras.Model,modelname,train_ds=train_ds,test_ds=test_ds,modelcheckpoint:bool = False,epochs= 10):
#     import os

#     #check if path is available
#     path = f'./model/models/{modelname}/{data_settings.stockname}/tensorboard/logs/fit'
#     pathlib.Path(path).mkdir(parents=True,exist_ok=True)

#     #tensorboard
#     log_dir =f'./model/models/{modelname}/{data_settings.stockname}/tensorboard/logs/fit/{datetime.datetime.now().strftime("(%Y-%m-%d)-(%H-%M-%S)")}'
#     tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#     #modelcheckpoint
#     base_path = f"./model/models/{modelname}/"
#     checkpoint_path = base_path + "tmp/cp-{epoch:04d}.ckpt"
#     checkpoint_dir = os.path.dirname(checkpoint_path)

#     model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
#         filepath=checkpoint_path,
#         monitor='val_loss',
#         verbose=1,
#         save_weights_only=True,
#         save_best_only=False
#     )

#     model.save_weights(checkpoint_path.format(epoch=0))


#     model.compile(loss='categorical_crossentropy',
#                     optimizer='adam',
#                     metrics=["categorical_accuracy","categorical_crossentropy"],
#                 )

#     if modelcheckpoint:
#         history = model.fit(
#                             train_ds,
#                             validation_data= test_ds,
#                             shuffle=True,
#                             epochs=epochs,
#                             verbose=1,
#                             callbacks=[tensorboard_callback,model_checkpoint_callback],
#                             use_multiprocessing=True,
#                             )
#     else:
#         history = model.fit(    
#                             train_ds,
#                             validation_data=test_ds,
#                             shuffle=True,
#                             epochs=epochs,
#                             verbose=1,
#                             callbacks=[tensorboard_callback],
#                             use_multiprocessing=True,
#                            )
    
#     return history

#TF Model
# input_layer = layers.Input(shape=(window_size,num_features))
# x = TCN(
#     nb_filters=128,
#     # kernel_size=3,
#     # nb_stacks=1,
#     # padding="causal",
#     # activation="swish",
#     use_batch_norm=True,
#     # return_sequences=True
#     )(input_layer)

# # x = layers.Flatten()(x)
# x = layers.Dense(3,activation="softmax")(x)


# model = keras.Model(
#     inputs = input_layer,
#     outputs = x
# )

# input_layer = layers.Input(shape=(window_size,num_features))

# x = layers.Dense(1024,activation="relu")(input_layer)
# x = layers.Dense(512,activation="relu")(x)
# x = layers.Dense(256,activation="relu")(x)
# x = layers.Dense(128,activation="relu")(x)
# x = layers.Dense(64,activation="relu")(x)
# x = layers.Dense(32,activation="relu")(x)
# x = layers.Dense(16,activation="relu")(x)
# x = layers.Flatten()(x)
# x = layers.Dense(3,activation="softmax")(x)


# model = keras.Model(
#     inputs = input_layer,
#     outputs = x
# )
# model.summary()
# compile_and_fit(
#     model=model,
#     modelname="simple_dense_first_try",
#     train_ds=train_ds,
#     test_ds=test_ds,
#     modelcheckpoint=True
# )

In [ ]:
# Normalizing with z-score https://developers.google.com/machine-learning/data-prep/transform/normalization

# train_df.mean().to_csv(f"{data_settings.base_path}/data/csv/train_mean.csv")
# train_df.std().to_csv(f"{data_settings.base_path}/data/csv/train_std.csv")
# train_df = (train_df - train_df.mean())/train_df.std()
# test_df = (test_df - train_df.mean())/train_df.std()

# Corrilation of buy and sell and do nothing with other rows of dataframes
# pd.options.display.max_rows =1000

# train_df.corr()["buy_4"]
# train_df.corr()["sell_4"]
# train_df.corr()["do_nothing"]

# import matplotlib.pyplot as plt

# for column in df.columns:
    
#     hist = df[column].hist(bins=100)
#     plt.savefig(f"./tmp/{column}_normalized.png")
#     hist.clear()


In [ ]:
# from lightgbm import LGBMClassifier

# lgbm_model = LGBMClassifier(
#     n_estimators = 1_000,
#     objective = "multiclass",
#     random_state = 23,
#     learning_rate = .01,
#     num_leaves = 128
# )

# lgbm_model.fit(
#     X = X_train,
#     y = y_train
    
# )
# from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

# p_train_lgbm = lgbm_model.predict(X_train)
# p_test_lgbm = lgbm_model.predict(X_test)

# acc_train_lgbm = accuracy_score(p_train,y_train)
# acc_test_lgbm = accuracy_score(p_test,y_test)

# rep_train_lgbm = classification_report(y_train,p_train)
# rep_test_lgbm = classification_report(y_test,p_test)

# con_train_lgbm = confusion_matrix(y_train,p_train)
# con_test_lgbm = confusion_matrix(y_test,p_test)

# print(rep_train_lgbm)
# print(rep_test_lgbm)
